In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
vm_size = "STANDARD_D2_V2"

from azureml.core.compute import ComputeTarget, AmlCompute

# Choose a name for your CPU cluster
cpu_cluster_name = "cpucluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size=vm_size,
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)


In [1]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice, normal
import os

# Specify parameter sampler
ps = RandomParameterSampling( {
        "learning_rate": normal(10, 3),
        "keep_probability": uniform(0.05, 0.1),
        "batch_size": choice(16, 32, 64, 128)
    }
)### YOUR CODE HERE ###

# Specify a Policy
policy = BanditPolicy(evaluation_interval=1, slack_factor=0.2, slack_amount=None, delay_evaluation=0)### YOUR CODE HERE ###

###if "training" not in os.listdir():###
   ### os.mkdir("./training")###

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory='.', compute_target=cpu_cluster, 
                vm_size="STANDARD_V2_D2", vm_priority='lowpriority', entry_script='train.py')### YOUR CODE HERE ###

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                                hyperparameter_sampling=ps,
                                policy=policy,
                                primary_metric_name='ACCURACY',
                                primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                max_total_runs=4,
                                max_concurrent_runs=4)
                                ### YOUR CODE HERE ###

ModuleNotFoundError: No module named 'azureml'

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
###HyperDriveRun(experiment, run_id, hyperdrive_config=hyperdrive_config)
hyperdrive_run = exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()

hyperdrive_run.get_status()
hyperdrive_run.wait_for_completion(show_output=True)

In [ ]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['Arguments']

print('Best run ID: ', best_run.id)
RunDetails(hyperdrive_run).show()

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
from azureml.core.dataset import Dataset
url = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
dataset2 = TabularDatasetFactory.from_delimited_files(path = url)
ds2 = dataset2.to_pandas_dataframe() ### YOUR CODE HERE ###


In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds2) ### your code###


In [ ]:
### your code here###
from sklearn.model_selection import train_test_split

import pandas as pd

x_train, x_test, y_train, y_test = train_test_split(x, y)

df_train = pd.concat([x_train, y_train], axis=1)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='regression',
    primary_metric='Accuracy',
    training_data=df_train,
    label_column_name='y',
    n_cross_validations=4)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###
from azureml.core.experiment import Experiment

remote_run = experiment.submit(automl_config, show_output=False)
remote_run

remote_run.wait_for_completion()

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_run, fitted_model = run.get_output()
fitted_model.steps